# TRANSFORMACIÓN DE DATOS

## IMPORTAR PAQUETES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

#Automcompletar rápido
%config IPCompleter.greedy=True

## IMPORTAR LOS DATOS

1.- Sustituir la ruta del proyecto.

In [2]:
ruta_proyecto = 'E:/alex/disco duro viejo/Desarrollo profesional/Python/DS4B/EstructuraDirectorio/03_MACHINE_LEARNING/08_CASOS/01_LEADSCORING'

2.- Nombrar los ficheros de datos.

In [3]:
nombre_cat = 'cat_resultado_eda.pickle'
nombre_num = 'num_resultado_eda.pickle'

3.- Cargar los datos.

In [4]:
cat = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_cat)
num = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_num)

4.- Separar la target.

In [5]:
target = num[['compra']].copy().reset_index(drop=True)

## TRANSFORMACIÓN DE CATEGÓRICAS

Aplicamos One Hot Encoding porque las variables categóricas que tenemos son todas nominales. Quizá puede haber dudas con 'ocupación' pero un estudiante podría tener más poder adquisitivo que un desempleado por lo que no se puede establecer claramente un orden.

### One Hot Encoding

#### Variables a aplicar OHE

In [6]:
var_ohe = ['origen','fuente','ult_actividad','ambito','ocupacion','descarga_lm']

#### Instanciar

In [7]:
ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')

#### Entrenar y aplicar

In [8]:
cat_ohe = ohe.fit_transform(cat[var_ohe])

#### Guardar como dataframe

In [9]:
cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out())

In [10]:
cat_ohe

,origen_API,origen_Landing Page Submission,origen_Lead Add Form,origen_OTROS,fuente_Chat,fuente_Direct Traffic,fuente_Google,fuente_OTROS,fuente_Organic Search,fuente_Reference,...,ambito_Operations Management,ambito_Select,ambito_Supply Chain Management,ambito_Travel and Tourism,ocupacion_OTROS,ocupacion_Student,ocupacion_Unemployed,ocupacion_Working Professional,descarga_lm_No,descarga_lm_Yes
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4804,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4805,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4806,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4807,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


## TRANSFORMACIÓN DE NUMÉRICAS

## UNIFICAR DATASETS TRANSFORMADOS

In [11]:
# Los indices de los dos dfs no coinciden, una alternativa es resetear el índice de 'num'
df = pd.concat([cat_ohe, num.reset_index()], axis = 1)

## REESCALAR VARIABLES

El reescalado si es importante porque el algoritmo que vamos a usar para la segmentación es un Kmedias y es sensible a los outliers (atípicos). Como hemos corregido los atípicos en la fase de calidad de datos no es necesario aplicar robust scaler y como vamos usar OneHotEncoding tiene más sentido aplicar MinMaxScaler ya que nos devolverá valores entre 0 y 1. StandardScaler ya sabemos que podría devolvernos valores entre -4 y 4

### Con Min-Max

#### Variables a reescalar con Min-Max

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 44 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   origen_API                                4809 non-null   float64
 1   origen_Landing Page Submission            4809 non-null   float64
 2   origen_Lead Add Form                      4809 non-null   float64
 3   origen_OTROS                              4809 non-null   float64
 4   fuente_Chat                               4809 non-null   float64
 5   fuente_Direct Traffic                     4809 non-null   float64
 6   fuente_Google                             4809 non-null   float64
 7   fuente_OTROS                              4809 non-null   float64
 8   fuente_Organic Search                     4809 non-null   float64
 9   fuente_Reference                          4809 non-null   float64
 10  ult_actividad_Chat Conversation     

In [13]:
var_mms = df.iloc[:,39:].columns

#### Instanciar

In [14]:
mms = MinMaxScaler()

#### Entrenar y aplicar

In [15]:
df_mms = mms.fit_transform(df[var_mms])

#### Guardar como dataframe

In [16]:
#Añadir sufijos a los nombres
nombres_mms = [variable + '_mms' for variable in var_mms]

#Guardar como dataframe
df_mms = pd.DataFrame(df_mms,columns = nombres_mms)

## UNIFICAR DATASETS REESCALADOS

### Unir todos los dataframes en el tablón analítico

In [17]:
id = df.id

In [18]:
df_tablon = pd.concat([id, cat_ohe, df_mms, target], axis = 1).set_index('id')

In [19]:
df_tablon

,origen_API,origen_Landing Page Submission,origen_Lead Add Form,origen_OTROS,fuente_Chat,fuente_Direct Traffic,fuente_Google,fuente_OTROS,fuente_Organic Search,fuente_Reference,...,ocupacion_Unemployed,ocupacion_Working Professional,descarga_lm_No,descarga_lm_Yes,visitas_total_mms,tiempo_en_site_total_mms,paginas_vistas_visita_mms,score_actividad_mms,score_perfil_mms,compra
id,,,,,,,,,,,,,,,,,,,,,
660728,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.092593,0.296655,0.104167,0.7,0.444444,0
660719,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.018519,0.134243,0.041667,0.5,0.666667,0
660681,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.037037,0.628521,0.041667,0.7,0.777778,1
660673,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.037037,0.721831,0.083333,0.6,1.000000,1
660664,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.7,0.444444,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579622,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.240741,0.620158,0.108333,0.6,0.555556,1
579615,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.092593,0.092430,0.104167,0.6,1.000000,1
579546,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.037037,0.104754,0.083333,0.6,0.888889,0


## GUARDAR DATASET TRAS TRANSFORMACIÓN DE DATOS

En formato pickle para no perder las modificaciones de metadatos.

In [20]:
#Definir los nombres del archivo
ruta_df_tablon = ruta_proyecto + '/02_Datos/03_Trabajo/' + 'df_tablon.pickle'

In [22]:
#Guardar los archivos
df_tablon.to_pickle(ruta_df_tablon)